*Copyright (c) Microsoft Corporation. All rights reserved.*

*Licensed under the MIT License.*

# Transformers BERT モデル (PyTorch) による日本語文章のテキスト分類

## 1. 事前準備

### 1.1 ライブラリのインポート

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import json
import os
import sys
from tempfile import TemporaryDirectory

import numpy as np
import pandas as pd
import scrapbook as sb
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

# # path の追加
# sys.path.append('../..')

from utils_nlp.common.timer import Timer
from utils_nlp.common.pytorch_utils import dataloader_from_dataset
from utils_nlp.dataset.multinli import load_pandas_df
from utils_nlp.models.transformers.sequence_classification import (
    Processor, SequenceClassifier)

from azureml.core import Workspace, Datastore, Dataset

# 表示する列データの幅を変更
pd.set_option("display.max_colwidth", 1000)

In [ ]:
# 念のため Transformer Version 確認
import transformers
transformers.__version__

### 1.2 モデルの設定
本ノートブックでは日本語対応BERTモデルのファインチューニングと評価を行います。

ここでは、[Hugging Face's PyTorch implementation](https://github.com/huggingface/transformers) をラップした [sequence classifier](../../utils_nlp/models/transformers/sequence_classification.py)を利用します。本コードでは、**bert-base-japanase-whole-word-masking** という学習済みモデルを利用します。 

In [ ]:
# パラメータ
DATA_FOLDER = TemporaryDirectory().name
CACHE_DIR = TemporaryDirectory().name
NUM_EPOCHS = 10
BATCH_SIZE = 16
NUM_GPUS = 2
MAX_LEN = 100
TRAIN_DATA_FRACTION =1 #サンプリングする場合は割合(<1)を指定
TEST_DATA_FRACTION =1 #サンプリングする場合は割合(<1)を指定
TRAIN_SIZE = 0.75
LABEL_COL = "label" # ラベルを含む列名
TEXT_COL = "text" # テキストを含む列名
MODEL_NAMES = ["bert-base-japanese-whole-word-masking"] #利用するモデル

### 1.3 データ準備
#### ダウンロード
[Livedoor ニュースコーパス](https://www.rondhuit.com/download/ldcc-20140209.tar.gz)をダウンロードして利用します。
<!-- データのダウンロードと加工手順は [bert-japanese](https://github.com/yoheikikuta/bert-japanese/) を参考にしています。 -->

In [ ]:
# from urllib.request import urlretrieve
# import tarfile

# text_url = "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"
# file_path = "./ldcc-20140209.tar.gz"
# urlretrieve(text_url, file_path)

In [ ]:
# # gz ファイルを解凍します。
# with tarfile.open('./ldcc-20140209.tar.gz', 'r:gz') as tar:
#     tar.extractall(path='livedoor')
#     tar.close() 

#### Pandas へのロード

In [ ]:
columns = ['url', 'date', 'label', 'title', 'text']
df = pd.DataFrame(columns = columns)
#df.set_index('url',inplace=True)

In [ ]:
path = "livedoor/text"

In [ ]:
for folder_name in os.listdir(path):
    print(folder_name)
    if folder_name.endswith(".txt") :
        continue
    for file in os.listdir(os.path.join(path, folder_name)):
        if folder_name == "LICENSE.txt" :
            continue
        with open(os.path.join(path, folder_name, file), 'r') as f:
            lines = f.read().split('\n')
            if len(lines) == 1:
                continue
            url = lines[0]
            date = lines[1]
            label = folder_name
            title = lines[3]
            text = "".join(lines[4:])
            data = {'url': url, 'date':date, 'label': label, 'title':title, 'text':text}
        s = pd.Series(data)        
        df = df.append(s, ignore_index=True)

In [ ]:
df.to_csv("livedoor-corpus.csv", index=False)

### 1.4 Azure Machine Learning ワークスペース接続

In [ ]:
ws = Workspace.from_config()
print(ws)

In [ ]:
# #テナントIDを指定する方法
# from azureml.core.authentication import InteractiveLoginAuthentication
# interactive_auth = InteractiveLoginAuthentication(tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47")
# ws = Workspace.from_config(auth=interactive_auth)
# print(ws)

### 1.5 Azure Machine Learning データセット登録と準備

In [ ]:
datastore = ws.get_default_datastore()

In [ ]:
datastore.upload_files(files=['livedoor-corpus.csv'],
                       target_path='livedoor-corpus',
                       overwrite=True,
                       show_progress=False)

In [ ]:
datastore_path = [(datastore, 'livedoor-corpus/livedoor-corpus.csv')]

In [ ]:
livedoor_ds = Dataset.Tabular.from_delimited_files(path=datastore_path)

In [ ]:
livedoor_ds.register(workspace=ws, name='livedoor',description='livedoor corpus', create_new_version = True)

In [ ]:
dataset = Dataset.get_by_name(ws, name='livedoor')
df = dataset.to_pandas_dataframe()

欠損値を除外します

In [ ]:
df = df[pd.isna(df["text"])==False]

In [ ]:
df.head()

本データセットでは9種類のラベルに分類されます。それぞれのデータ数を確認します。

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 5))
sns.countplot(df[LABEL_COL])

データを学習用、テスト用に分割します。またラベル名をエンコーディングしてBERTで扱えるようにします。

In [ ]:
# データ分割
df_train, df_test = train_test_split(df, train_size = TRAIN_SIZE, random_state=0)

In [ ]:
# # 必要であればサンプリング
# df_train = df_train.sample(frac=TRAIN_DATA_FRACTION).reset_index(drop=True)
# df_test = df_test.sample(frac=TEST_DATA_FRACTION).reset_index(drop=True)

In [ ]:
# ラベルのエンコーディング
label_encoder = LabelEncoder()
df_train[LABEL_COL] = label_encoder.fit_transform(df_train[LABEL_COL])
df_test[LABEL_COL] = label_encoder.transform(df_test[LABEL_COL])

num_labels = len(np.unique(df_train[LABEL_COL]))

In [ ]:
print("Number of unique labels: {}".format(num_labels))
print("Number of training examples: {}".format(df_train.shape[0]))
print("Number of testing examples: {}".format(df_test.shape[0]))

## 2. モデル学習と評価
### 2.1 学習済みモデルの選択

[Hugging Face](https://github.com/huggingface/transformers) には学習済みモデルが公開されており簡単に利用することができます。テキスト分類で利用できるモデル一覧を出力します。

In [ ]:
pd.DataFrame({"モデル名": SequenceClassifier.list_supported_models()})

### 2.2 ファインチューニング

本コードで実装されているラッパーを利用することで簡単にファインチューニングが実行できます。

In [ ]:
# 利用するモデル名
print(MODEL_NAMES)

データ前処理、ファインチューニング、テストデータの予測、評価をステップを実施していきます。

In [ ]:
results = {}

model_name = MODEL_NAMES[0]

# 前処理
processor = Processor(
    model_name=model_name,
    to_lower=model_name.endswith("uncased"),
    cache_dir=CACHE_DIR,
)

In [ ]:
train_dataset = processor.dataset_from_dataframe(
    df_train, TEXT_COL, LABEL_COL, max_len=MAX_LEN
)
train_dataloader = dataloader_from_dataset(
    train_dataset, batch_size=BATCH_SIZE, num_gpus=NUM_GPUS, shuffle=True
)
test_dataset = processor.dataset_from_dataframe(
    df_test, TEXT_COL, LABEL_COL, max_len=MAX_LEN
)
test_dataloader = dataloader_from_dataset(
    test_dataset, batch_size=BATCH_SIZE, num_gpus=NUM_GPUS, shuffle=False
)

In [ ]:
# ファインチューニング
classifier = SequenceClassifier(
    model_name=model_name, num_labels=num_labels, cache_dir=CACHE_DIR
)

In [ ]:
with Timer() as t:
    classifier.fit(
        train_dataloader, num_epochs=NUM_EPOCHS, num_gpus=NUM_GPUS, verbose=False,
    )
train_time = t.interval / 3600

In [ ]:
# テストデータの予測
preds = classifier.predict(test_dataloader, num_gpus=NUM_GPUS, verbose=False)

# 評価
accuracy = accuracy_score(df_test[LABEL_COL], preds)
class_report = classification_report(
    df_test[LABEL_COL], preds, target_names=label_encoder.classes_, output_dict=True
)

# 結果の保存
results[model_name] = {
    "accuracy": accuracy,
    "f1-score": class_report["macro avg"]["f1-score"],
    "time(hrs)": train_time,
}

In [ ]:
df_test[LABEL_COL] = label_encoder.inverse_transform(df_test[LABEL_COL])
df_test["pred"] = label_encoder.inverse_transform(preds)

In [ ]:
df_test.head(1)

### 2.3 評価

精度、F1-スコア、学習時間を確認します。

In [ ]:
df_results = pd.DataFrame(results)
df_results

In [ ]:
# for testing
sb.glue("accuracy", df_results.iloc[0, :].mean())
sb.glue("f1", df_results.iloc[1, :].mean())